import psycopg2
import json

# Database credentials
db_host = "localhost"
db_name = "your_database"
db_user = "your_user"
db_password = "your_password"
db_port = 5432

# JSON file path
json_file = "games.json"

try:
    # Connect to PostgreSQL
    conn = psycopg2.connect(host=db_host, database=db_name, user=db_user, password=db_password, port=db_port)
    cur = conn.cursor()

    # Load JSON data from file
    with open(json_file, 'r', encoding='utf-8') as f:
        games_data = json.load(f)

    # Iterate through the JSON data and insert into the table
    for game_id, game_data in games_data.items():
        try:
            # Convert the Python dictionary to a JSON string and cast to JSONB
            sql_insert = "INSERT INTO steam_games (id, game_data) VALUES (%s, %s::JSONB)"
            cur.execute(sql_insert, (game_id, json.dumps(game_data)))  # Pass the JSON string as a tuple

        except Exception as e:
            print(f"Error inserting game with ID: {game_id}. Error: {e}")  # Print the game ID
            conn.rollback() # Rollback the transaction if there's an error

    # Commit the changes
    conn.commit()
    print(f"Successfully inserted data into the database.")

except psycopg2.Error as e:
    print(f"Database error: {e}")
    if conn:
        conn.rollback() # Rollback the transaction in case of a database error

except FileNotFoundError:
    print(f"Error: File not found: {json_file}")

except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")

except Exception as e:
    print(f"An unexpected error occurred: {e}")

finally:
    # Close connection
    if conn:
        cur.close()
        conn.close()